In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("retailrocket/ecommerce-dataset")

print("Path to dataset files:", "D:\\Pallav-Task_MKS_Digitech_LLP\\Assignment 2")


Path to dataset files: D:\Pallav-Task_MKS_Digitech_LLP\Assignment 2


In [2]:
!kaggle datasets download -d retailrocket/ecommerce-dataset


Dataset URL: https://www.kaggle.com/datasets/retailrocket/ecommerce-dataset
License(s): CC-BY-NC-SA-4.0
ecommerce-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
import os

# Replace "ecommerce-dataset.zip" with the actual name of the downloaded zip file
with zipfile.ZipFile("ecommerce-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/ecommerce-dataset")

# List the files in the extracted directory
os.listdir("/content/ecommerce-dataset")


['category_tree.csv',
 'events.csv',
 'item_properties_part1.csv',
 'item_properties_part2.csv']

In [4]:
import pandas as pd

In [5]:
!pip install cudf


  Using cached cudf-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cudf
  Running setup.py clean for cudf
Failed to build cudf
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cudf)


In [6]:
pip install cudf


  Using cached cudf-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cudf
  Running setup.py clean for cudf
Failed to build cudf
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cudf)


In [7]:
!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [8]:
import dask.dataframe as dd

# Load datasets
events = dd.read_csv('/content/ecommerce-dataset/events.csv')
item_properties = dd.read_csv('/content/ecommerce-dataset/item_properties_part1.csv')
category_tree = dd.read_csv('/content/ecommerce-dataset/category_tree.csv')

/usr/local/lib/python3.10/dist-packages/dask/array/chunk_types.py:110: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  import cupy


In [9]:
events.columns

Index(['timestamp', 'visitorid', 'event', 'itemid', 'transactionid'], dtype='object')

In [10]:
item_properties.columns

Index(['timestamp', 'itemid', 'property', 'value'], dtype='object')

In [11]:
category_tree.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [12]:
events_item_properties = events.merge(item_properties, on='itemid', how='left')

In [13]:
# Persist the dataframe to optimize the operations
events_item_properties = events_item_properties.persist()

# Convert 'value' to numeric, coercing errors to NaN
events_item_properties['value_numeric'] = dd.to_numeric(events_item_properties['value'], errors='coerce')

# Drop rows where 'value_numeric' is NaN (non-numeric values)
events_item_properties = events_item_properties.dropna(subset=['value_numeric'])

# Ensure 'value_numeric' is integer type
events_item_properties['value_numeric'] = events_item_properties['value_numeric'].astype('int32')

# Compute and show the first few rows of the result
print(events_item_properties.head())


      timestamp_x  visitorid event  itemid  transactionid   timestamp_y  \
4   1433220899221     824915  view  428805            NaN  1.439694e+12   
6   1433220899221     824915  view  428805            NaN  1.435460e+12   
8   1433220899221     824915  view  428805            NaN  1.432436e+12   
14  1433220899221     824915  view  428805            NaN  1.431227e+12   
28  1433220899221     824915  view  428805            NaN  1.433646e+12   

      property   value  value_numeric  
4          202  916731         916731  
6          917  873834         873834  
8   categoryid     646            646  
14           6  726714         726714  
28         112  679677         679677  


In [14]:
final_df = events_item_properties.merge(
    category_tree,
    left_on='value_numeric',
    right_on='categoryid',
    how='left'
)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:520: UserWarning: Merging dataframes with merge column data type mismatches: 
+---------------------------------+------------+-------------+
| Merge columns                   | left dtype | right dtype |
+---------------------------------+------------+-------------+
| ('value_numeric', 'categoryid') | int32      | int64       |
+---------------------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


In [15]:
final_df.columns

Index(['timestamp_x', 'visitorid', 'event', 'itemid', 'transactionid',
       'timestamp_y', 'property', 'value', 'value_numeric', 'categoryid',
       'parentid'],
      dtype='object')

In [16]:
# feature engineering

import numpy as np

np.random.seed(42)

In [ ]:
import dask.dataframe as dd
import numpy as np

# Assuming final_df is a Dask DataFrame
user_measurements = final_df[['visitorid']].drop_duplicates().reset_index(drop=True)

# Generate random heights and weights using Dask's assign method
user_measurements = user_measurements.assign(
    user_height=lambda x: np.random.randint(150, 200, size=len(x)),
    user_weight=lambda x: np.random.randint(50, 100, size=len(x))
)


In [ ]:
final_df = final_df.merge(user_measurements, on='visitorid', how='left')

In [ ]:
# If final_df is a Dask DataFrame, no need to use compute() within the lambda
final_df = final_df.assign(
    size=lambda x: np.random.choice(['S', 'M', 'L', 'XL'], size=len(x))
)

# If final_df is a Dask DataFrame, and you want to persist it:
final_df = final_df.persist()

In [ ]:
final_df.columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
label_encoder = LabelEncoder()
final_df['size_encoded'] = label_encoder.fit_transform(final_df['size'])

In [ ]:
# Features and target
X = final_df[['user_height', 'user_weight', 'categoryid']]
y = final_df['size_encoded']

In [ ]:
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
# Build the Neural Network
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer for multi-class classification
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)


In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes, average='weighted')

In [ ]:
print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")

# Decode the predicted sizes
predicted_sizes = label_encoder.inverse_transform(y_pred_classes)
print("Predicted Sizes:", predicted_sizes[:10])

In [ ]:
# Save the model as an .h5 file
model.save('size_recommendation_model.h5')

# Load the model
from tensorflow.keras.models import load_model

loaded_model = load_model('size_recommendation_model.h5')

# Verify the loaded model
loaded_model.summary()

In [ ]:
#user profiling

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# Create user-item interaction matrix
interaction_matrix = final_df.pivot_table(index='visitorid', columns='itemid', values='transactionid', aggfunc='count', fill_value=0)
print("Interaction Matrix:")
print(interaction_matrix.head())

In [ ]:
#Collaborative Filtering

# Compute similarity between users using Cosine Similarity
user_similarity = cosine_similarity(interaction_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

print("User Similarity Matrix:")
print(user_similarity_df.head())

# Recommend products for a given user
def recommend_items_for_user(user_id, num_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:num_recommendations + 1]

    # Get items viewed/purchased by similar users
    recommended_items = set()
    for similar_user in similar_users:
        items = interaction_matrix.loc[similar_user][interaction_matrix.loc[similar_user] > 0].index
        recommended_items.update(items)

    return list(recommended_items)

print("Recommended Items for User 1001:", recommend_items_for_user(1001))


In [ ]:
#Content-Based Filtering
# Use item-based features: category, size, etc.
item_features = final_df.groupby('itemid')[['categoryid']].first()

# Normalize and calculate similarity between items
item_similarity = cosine_similarity(item_features)
item_similarity_df = pd.DataFrame(item_similarity, index=item_features.index, columns=item_features.index)

print("Item Similarity Matrix:")
print(item_similarity_df.head())

def recommend_similar_items(item_id, num_recommendations=5):
    similar_items = item_similarity_df[item_id].sort_values(ascending=False).index[1:num_recommendations + 1]
    return similar_items.tolist()

print("Recommended Items Similar to Item 1:", recommend_similar_items(1))


In [ ]:
#Hybrid Recommendation System

def hybrid_recommendation(user_id, item_id, num_recommendations=5):
    user_based_recommendations = recommend_items_for_user(user_id, num_recommendations)
    item_based_recommendations = recommend_similar_items(item_id, num_recommendations)

    # Combine recommendations (e.g., weighted combination)
    combined_recommendations = set(user_based_recommendations).union(set(item_based_recommendations))
    return list(combined_recommendations)[:num_recommendations]

print("Hybrid Recommendations for User 1001 and Item 1:", hybrid_recommendation(1001, 1))


In [ ]:
import numpy as np

ground_truth = {
    1001: [1, 2, 5],
    1002: [3, 4],
    1003: [2, 6],
}

# Recommendations for each user (simulated)
recommendations = {
    1001: [1, 3, 5, 7],
    1002: [3, 8, 9],
    1003: [6, 2, 10, 11],
}

# Precision@K
def precision_at_k(recommended, relevant, k):
    recommended_at_k = recommended[:k]
    relevant_and_recommended = set(recommended_at_k) & set(relevant)
    return len(relevant_and_recommended) / k

# Recall@K
def recall_at_k(recommended, relevant, k):
    recommended_at_k = recommended[:k]
    relevant_and_recommended = set(recommended_at_k) & set(relevant)
    return len(relevant_and_recommended) / len(relevant)

# Average Precision (AP) for a single user
def average_precision(recommended, relevant):
    relevant_and_recommended = 0
    sum_precisions = 0
    for i, item in enumerate(recommended, start=1):
        if item in relevant:
            relevant_and_recommended += 1
            sum_precisions += relevant_and_recommended / i
    return sum_precisions / len(relevant) if relevant else 0

# Mean Average Precision (MAP) across all users
def mean_average_precision(ground_truth, recommendations):
    return np.mean([average_precision(recommendations[user], ground_truth[user])
                    for user in ground_truth])

# Evaluation
K = 3
precision_scores = {user: precision_at_k(recommendations[user], ground_truth[user], K) for user in ground_truth}
recall_scores = {user: recall_at_k(recommendations[user], ground_truth[user], K) for user in ground_truth}
map_score = mean_average_precision(ground_truth, recommendations)

print("Precision@K for each user:", precision_scores)
print("Recall@K for each user:", recall_scores)
print(f"Mean Average Precision (MAP): {map_score:.2f}")
